Para empezar a utilizar mongodb desde 0, vamos a partir realizando los siguientes pasos:


1.   Instalaremos $\texttt{mongodb}$
2.   Inicializaremos el proceso $\texttt{mongodb}$
3.   Instalaremos el comando $\texttt{gdown}$ para bajar la base de datos en formato json de google drive.
4.   Importaremos los datos por linea de comando. Dejaremos los datos en la base de datos $\texttt{local}$, en la colección $\texttt{pokedex}$, y le informaremos al comando que los datos vienen como una arreglo de jsons.




In [1]:

!apt update
!apt install wget curl gnupg2 software-properties-common apt-transport-https ca-certificates lsb-release
!curl -fsSL https://www.mongodb.org/static/pgp/server-6.0.asc|sudo gpg --dearmor -o /etc/apt/trusted.gpg.d/mongodb-6.gpg
!echo "deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu $(lsb_release -cs)/mongodb-org/6.0 multiverse" | tee /etc/apt/sources.list.d/mongodb-org-6.0.list
!apt update
!apt install mongodb-org
!mkdir /data
!mkdir /data/db
!mongod --fork --logpath /var/log/mongodb/mongod.log

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,750 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,059 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,036

In [2]:

!gdown https://drive.google.com/uc?id=108aLE4mFE5bZYitFfX5c14Qj_aYA5MO_
!mongoimport --db local --collection pokedex --jsonArray < pokedex.json

Downloading...
From: https://drive.google.com/uc?id=108aLE4mFE5bZYitFfX5c14Qj_aYA5MO_
To: /content/pokedex.json
100% 2.95M/2.95M [00:00<00:00, 109MB/s]
2025-06-24T21:57:21.300+0000	connected to: mongodb://localhost/
2025-06-24T21:57:22.046+0000	809 document(s) imported successfully. 0 document(s) failed to import.


A continuación importaremos la librería de mongo para python, crearemos un cliente que se conectara a $\texttt{localhost}$ por defecto, nos conectaremos a la base local, y luego a la colección pokedex la cual almacenaremos en la variable $\texttt{collection}$.

In [3]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 15.6 MB/s eta 0:00:00


In [4]:
from pymongo import MongoClient
client = MongoClient()
client.list_database_names() # ['admin', 'local']
db = client['local']
collection = db['pokedex']

Para partir, podemos listar toda la colección usando el comando $\texttt{collection.find({})}$. Para no saturar la salida, nos limitaremos a mirar el primero de los registros.

In [5]:
import pprint
for pokemon in collection.find({}).limit(1):
    pprint.pprint(pokemon)

{'Abilities': ['Overgrow', 'Chlorophyll'],
 'Attack': 49,
 'Capture Rate': 45,
 'Category': 'Seed Pokémon',
 'Defense': 49,
 'Egg Steps': 5120,
 'Exp Group': 'Medium Slow',
 'HP': 45,
 'Height (ft)': '2\'04"',
 'Height (m)': 0.7,
 'Id': '001',
 'Moves': {'Double-Edge': {'Accuracy': 100,
                           'Description': 'A reckless, life-risking tackle. '
                                          'This also damages the user quite a '
                                          'lot.',
                           'Effect %': '--',
                           'Level': 27,
                           'PP': 15,
                           'Power': 120,
                           'Type': 'Normal'},
           'Growl': {'Accuracy': 100,
                     'Description': 'The user growls in an endearing way, '
                                    'making opposing Pokémon less wary. This '
                                    'lowers their Attack stat.',
                     'Effect %': '--'

Fijense que aparte de los datos básicos del pokemon, también se presentan las habilidades (una lista de strings), y los movimientos (un arreglo asociativo/diccionario de los movimientos y sus detalles).

Si queremos ahora listar los pokemones que se llaman $\texttt{Charmander}$ podemos hacer la siguiente consulta:

In [6]:
for pokemon in collection.find({'Name': 'Charmander'}):
    pprint.pprint(pokemon)

{'Abilities': ['Blaze', 'Solar Power'],
 'Attack': 52,
 'Capture Rate': 45,
 'Category': 'Lizard Pokémon',
 'Defense': 43,
 'Egg Steps': 5120,
 'Exp Group': 'Medium Slow',
 'HP': 39,
 'Height (ft)': '2\'00"',
 'Height (m)': 0.6,
 'Id': '004',
 'Moves': {'Dragon Rage': {'Accuracy': 100,
                           'Description': 'This attack hits the target with a '
                                          'shock wave of pure rage. This '
                                          'attack always inflicts 40 HP '
                                          'damage.',
                           'Effect %': '--',
                           'Level': 16,
                           'PP': 10,
                           'Power': '??',
                           'Type': 'Dragon'},
           'Ember': {'Accuracy': 100,
                     'Description': 'The target is attacked with small flames. '
                                    'This may also leave the target with a '
                         

Y si queremos proyectar algunos de sus atributos lo hacemos tal como si fuera un diccionario:

In [7]:
for pokemon in collection.find({'Name': 'Mew'}):
    print(pokemon['Speed'],pokemon['Abilities'])

100 ['Synchronize']


Recomendamos hechar un vistazo [aquí](https://docs.mongodb.com/manual/tutorial/query-documents/) para ver otros ejemplos de consultas que utilizarán en las siguientes preguntas:

#Pregunta 1
Encuentre los nombre de los pokemons que pesen mas de 150kg.
[Ordene](https://www.w3schools.com/python/python_mongodb_sort.asp) los resultados por peso descendiente

In [8]:
for pokemon in collection.find({'Weight (kg)': {'$gt': 150}}).sort('Weight (kg)', -1):
    pprint.pprint(pokemon['Name'])

'Cosmoem'
'Celesteela'
'Groudon'
'Mudsdale'
'Guzzlord'
'Stakataka'
'Melmetal'
'Giratina'
'Dialga'
'Metagross'
'Avalugg'
'Snorlax'
'Heatran'
'Regigigas'
'Steelix'
'Wailord'
'Aggron'
'Kyogre'
'Zekrom'
'Probopass'
'Palkia'
'Buzzwole'
'Golurk'
'Reshiram'
'Kyurem'
'Arceus'
'Torterra'
'Zygarde'
'Golem'
'Hippowdon'
'Mamoswine'
'Rhyperior'
'Tyrantrum'
'Gigalith'
'Beartic'
'Terrakion'
'Glalie'
'Hariyama'
'Cobalion'
'Palossand'
'Gyarados'
'Regirock'
'Solgaleo'
'Necrozma'
'Aurorus'
'Lapras'
'Mantine'
'Camerupt'
'Lugia'
'Xerneas'
'Turtonator'
'Onix'
'Dragonite'
'Dhelmise'
'Rayquaza'
'Registeel'
'Yveltal'
'Metang'
'Tyranitar'
'Scolipede'
'Crustle'
'Virizion'
'Ho-Oh'
'Entei'
'Volcanion'
'Suicune'
'Bronzong'
'Drampa'
'Magnezone'
'Crabominable'
'Raikou'
'Regice'
'Lunatone'
'Milotic'
'Hydreigon'
'Arcanine'
'Solrock'
'Kingdra'
'Pupitar'
'Walrein'
'Goodra'


#Pregunta 2
Encuentre los nombres de los pokemones ''Glass cannon'', que son los que tienen ataque sobre 120 y defensa bajo 50.

In [9]:
for pokemon in collection.find({'Attack': {'$gt': 120}, 'Defense': {'$lt': 50}}):
    pprint.pprint(pokemon['Name'])

'Cranidos'
'Pheromosa'


#Pregunta 3
Encuentre los pokemones "defensivos", que son los que tienen ya sea defensa, hp, o defensa especial (sp. defense) mayor o igual a 120.

In [10]:
for pokemon in collection.find({'$or': [{'Defense': {'$gte': 120}},{'Hp': {'$gte': 120}}, {'Sp. Defense': {'$gte': 120}}]}):
    pprint.pprint(pokemon['Name'])

'Golem'
'Cloyster'
'Onix'
'Weezing'
'Rhydon'
'Omastar'
'Forretress'
'Steelix'
'Shuckle'
'Magcargo'
'Skarmory'
'Donphan'
'Lugia'
'Nosepass'
'Lairon'
'Aggron'
'Torkoal'
'Dusclops'
'Relicanth'
'Regirock'
'Registeel'
'Metagross'
'Groudon'
'Bastiodon'
'Rhyperior'
'Tangrowth'
'Gliscor'
'Leafeon'
'Probopass'
'Dusknoir'
'Uxie'
'Dialga'
'Cresselia'
'Giratina'
'Arceus'
'Gigalith'
'Crustle'
'Cofagrigus'
'Carracosta'
'Ferrothorn'
'Cobalion'
'Zekrom'
'Chesnaught'
'Doublade'
'Aegislash'
'Carbink'
'Gourgeist'
'Avalugg'
'Zygarde'
'Diancie'
'Volcanion'
'Toxapex'
'Golisopod'
'Pyukumuku'
'Turtonator'
'Kommo-o'
'Cosmoem'
'Buzzwole'
'Kartana'
'Stakataka'
'Melmetal'


#Pregunta 4
Seleccione los pokemones que tienen el movimiento $\texttt{Take Down}$ de nivel 15.

**Hint**: Puede consultar en diccionarios embebidos concatenando las llaves con un punto ".".

Por ejemplo, para un dato:
```
[{
  a: {
    b: 'foo'
  }
}]
```
se puede consultar por los documentos tal que $\texttt{b}$ sea $\texttt{foo}$, escribiendo $\texttt{collection.find({'a.b': 'foo'})}$.

In [11]:
for pokemon in collection.find({'Moves.Take Down.Level': 15}):
    pprint.pprint(pokemon['Name'])

'Bulbasaur'
'Venusaur'
'Ivysaur'
'Rampardos'
'Shieldon'
'Bastiodon'
'Cranidos'
'Lillipup'
'Herdier'
'Stoutland'
'Bunnelby'
'Diggersby'
'Amaura'
'Aurorus'
'Bergmite'
'Avalugg'


#Pregunta 5
Seleccione los pokemones que tienen el movimiento $\texttt{Take Down}$.

**Hint**: Puede testear si es que una llave existe utilizando [$exists](https://docs.mongodb.com/manual/reference/operator/query/exists/)

In [12]:
for pokemon in collection.find({'Moves.Take Down': { '$exists': True }}):
    pprint.pprint(pokemon['Name'])

'Bulbasaur'
'Venusaur'
'Ivysaur'
'Growlithe'
'Ponyta'
'Rapidash'
'Seel'
'Dewgong'
'Rhyhorn'
'Rhydon'
'Chansey'
'Tauros'
'Eevee'
'Aerodactyl'
'Noctowl'
'Chinchou'
'Lanturn'
'Hoothoot'
'Mareep'
'Flaaffy'
'Ampharos'
'Pineco'
'Forretress'
'Dunsparce'
'Qwilfish'
'Heracross'
'Swinub'
'Piloswine'
'Mantine'
'Phanpy'
'Stantler'
'Blissey'
'Mudkip'
'Marshtomp'
'Swampert'
'Poochyena'
'Mightyena'
'Zigzagoon'
'Aron'
'Lairon'
'Aggron'
'Carvanha'
'Numel'
'Camerupt'
'Swablu'
'Altaria'
'Chimecho'
'Luvdisc'
'Beldum'
'Metang'
'Relicanth'
'Metagross'
'Starly'
'Staraptor'
'Staravia'
'Bibarel'
'Bidoof'
'Rampardos'
'Shieldon'
'Bastiodon'
'Cranidos'
'Cherubi'
'Cherrim'
'Gible'
'Gabite'
'Garchomp'
'Hippopotas'
'Hippowdon'
'Mantyke'
'Rhyperior'
'Mamoswine'
'Tepig'
'Pignite'
'Emboar'
'Lillipup'
'Herdier'
'Stoutland'
'Audino'
'Basculin'
'Trubbish'
'Deerling'
'Sawsbuck'
'Karrablast'
'Larvesta'
'Cobalion'
'Virizion'
'Terrakion'
'Keldeo'
'Chespin'
'Quilladin'
'Chesnaught'
'Bunnelby'
'Litleo'
'Pyroar'
'Skiddo'
'Digger

#Pregunta 6
1. Seleccione los pokemones que tienen la habilidad $\texttt{Telepathy}$


In [13]:
for pokemon in collection.find({'Abilities': 'Telepathy'}):
    pprint.pprint(pokemon['Name'])

'Wobbuffet'
'Ralts'
'Gardevoir'
'Kirlia'
'Medicham'
'Meditite'
'Wynaut'
'Dialga'
'Palkia'
'Giratina'
'Musharna'
'Munna'
'Elgyem'
'Beheeyem'
'Noibat'
'Noivern'
'Oranguru'
'Tapu Koko'
'Tapu Lele'
'Tapu Fini'
'Tapu Bulu'


2. Seleccione los pokemones que tienen la habilidad $\texttt{Telepathy}$
 **y** $\texttt{Synchronize}$ (pueden tener mas habilidades aparte de estas dos.

   **Hint**: puede utilizar el operador [$all](https://docs.mongodb.com/manual/tutorial/query-arrays/) .

#Pregunta 7
Para la siguiente pregunta vamos a crear un índice sobre la columna $\texttt{Category}$.

In [14]:
collection.create_index([('Category', 'text')])

'Category_text'

Usando el operador [$text](https://docs.mongodb.com/manual/text-search/) responda a las siguientes consultas.
1.  Devuelva los nombres y categorías de los pokemons tales que su categoría contenga ya sea la palabra **Sea** o **Gas**.

In [15]:
for pokemon in collection.find({'$text': {'$search': 'Sea Gas'}}):
    print(pokemon['Name'], ":", pokemon['Category'])

Gastly : Gas Pokémon
Haunter : Gas Pokémon
Weezing : Poison Gas Pokémon
Koffing : Poison Gas Pokémon
Dhelmise : Sea Creeper Pokémon
Pyukumuku : Sea Cucumber Pokémon
Popplio : Sea Lion Pokémon
Oshawott : Sea Otter Pokémon
Phione : Sea Drifter Pokémon
Gastrodon : Sea Slug Pokémon
Shellos : Sea Slug Pokémon
Floatzel : Sea Weasel Pokémon
Buizel : Sea Weasel Pokémon
Kyogre : Sea Basin Pokémon
Gorebyss : South Sea Pokémon
Huntail : Deep Sea Pokémon
Lileep : Sea Lily Pokémon
Dewgong : Sea Lion Pokémon
Seel : Sea Lion Pokémon


2. Devuelva los nombres y categorías de los pokemons tales que su categoría contenga la palabra **Sea** y no **Deep**.

In [16]:
for pokemon in collection.find({'$text': {'$search': 'Sea -Deep'}}):
    print(pokemon['Name'], ":", pokemon['Category'])

Dhelmise : Sea Creeper Pokémon
Pyukumuku : Sea Cucumber Pokémon
Popplio : Sea Lion Pokémon
Oshawott : Sea Otter Pokémon
Phione : Sea Drifter Pokémon
Gastrodon : Sea Slug Pokémon
Shellos : Sea Slug Pokémon
Floatzel : Sea Weasel Pokémon
Buizel : Sea Weasel Pokémon
Kyogre : Sea Basin Pokémon
Gorebyss : South Sea Pokémon
Lileep : Sea Lily Pokémon
Dewgong : Sea Lion Pokémon
Seel : Sea Lion Pokémon


3. Devuelva los nombres y categorías de los pokemons tales que su categoría contenga ambas palabras **Sea** y **Lion**.

In [17]:
for pokemon in collection.find({'$text': {'$search': '\"Sea\" \"Lion\"'}}):
    print(pokemon['Name'], ":", pokemon['Category'])

Popplio : Sea Lion Pokémon
Dewgong : Sea Lion Pokémon
Seel : Sea Lion Pokémon
